# Intro
I still lack knowledge and implementation ability about deep-learning. 

Therefore, the following link was used to focus on understanding the code and the thesis<br/>
(https://www.youtube.com/watch?v=2Kawrd5szHE)

I'm sorry I didn't meet your expectations, and I'll try to explain what I understand and what I know.

The subject that I studied is **capsule-network**.

<br/>

# imports & load data

- import library to use and load mnist data

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


# input images and labels
- mnist data consisted of 28*28 size

In [43]:
X = tf.placeholder(tf.float32, shape=[None, 28,28,1])
y = tf.placeholder(shape=[None], dtype=tf.int64, name="y")

# CapsNet architecture

## primary architecture
CapsNet architecture (3 page of a paper) 
- Conv1 > filter : 256, kernel_size : 9, stride : 1, activation : relu
- Conv2 > filter : 32channel of convolutional 8d capsule, kernel_size : 9, stride :2, activation : relu

In [6]:
caps1_n_maps = 32
caps1_n_caps = caps1_n_maps * 6 * 6  # 1152 primary capsules(4page of paper)
caps1_n_dims = 8

In [7]:
conv1_params = {
    "filters": 256,
    "kernel_size": 9,
    "strides": 1,
    "padding": "valid",
    "activation": tf.nn.relu,
}

conv2_params = {
    "filters": caps1_n_maps * caps1_n_dims, # 256 convolutional filters
    "kernel_size": 9,
    "strides": 2,
    "padding": "valid",
    "activation": tf.nn.relu
}

In [8]:
# set parameter of conv2d layer 
conv1 = tf.layers.conv2d(X, name="conv1", **conv1_params)
conv2 = tf.layers.conv2d(conv1, name="conv2", **conv2_params)

In [57]:
# to reshape for fully connected to the digitcapsule
caps1_raw = tf.reshape(conv2, [-1, caps1_n_caps, caps1_n_dims],name="caps1_raw")

In [58]:
# implement of squshing(2page) 
def squash(s, axis=-1, epsilon=1e-7, name=None):
    with tf.name_scope(name, default_name="squash"):
        
        # to calucate norm
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,keep_dims=True)
        # Keep the denominator from falling to zero(by sum epsilon)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector

In [15]:
caps1_output = squash(caps1_raw, name="caps1_output")

## digits caps
- digit capsule 10capsule * 16dimension (figure2(4page))

In [24]:
caps2_n_caps = 10
caps2_n_dims = 16

Need $\mathbf{W}_{i,j}$ to caculate $\hat{\mathbf{u}}_{j|i} = \mathbf{W}_{i,j} \, \mathbf{u}_i$

$
\pmatrix{
  \mathbf{W}_{1,1} & \mathbf{W}_{1,2} & \cdots & \mathbf{W}_{1,10} \\
  \mathbf{W}_{2,1} & \mathbf{W}_{2,2} & \cdots & \mathbf{W}_{2,10} \\
  \vdots & \vdots & \ddots & \vdots \\
  \mathbf{W}_{1152,1} & \mathbf{W}_{1152,2} & \cdots & \mathbf{W}_{1152,10}
} \times
\pmatrix{
  \mathbf{u}_1 & \mathbf{u}_1 & \cdots & \mathbf{u}_1 \\
  \mathbf{u}_2 & \mathbf{u}_2 & \cdots & \mathbf{u}_2 \\
  \vdots & \vdots & \ddots & \vdots \\
  \mathbf{u}_{1152} & \mathbf{u}_{1152} & \cdots & \mathbf{u}_{1152}
}
=
\pmatrix{
\hat{\mathbf{u}}_{1|1} & \hat{\mathbf{u}}_{2|1} & \cdots & \hat{\mathbf{u}}_{10|1} \\
\hat{\mathbf{u}}_{1|2} & \hat{\mathbf{u}}_{2|2} & \cdots & \hat{\mathbf{u}}_{10|2} \\
\vdots & \vdots & \ddots & \vdots \\
\hat{\mathbf{u}}_{1|1152} & \hat{\mathbf{u}}_{2|1152} & \cdots & \hat{\mathbf{u}}_{10|1152}
}
$

first array is (1152, 10, 16, 8), and the shape of the second array is (1152, 10, 8, 1)

In [27]:
# I understand the general flow of this process,
# but I did not fully understand the function (tf.expand_dims,tf.tile)

init_sigma = 0.01

W_init = tf.random_normal(shape=(1, caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),stddev=init_sigma, dtype=tf.float32, name="W_init")
W = tf.Variable(W_init, name="W")

batch_size = tf.shape(X)[0]
W_tiled = tf.tile(W, [batch_size, 1, 1, 1, 1], name="W_tiled")

caps1_output_expanded = tf.expand_dims(caps1_output, -1, name="caps1_output_expanded")
caps1_output_tile = tf.expand_dims(caps1_output_expanded, 2, name="caps1_output_tile")
caps1_output_tiled = tf.tile(caps1_output_tile, [1, 1, caps2_n_caps, 1,1], name="caps1_output_tiled")
caps2_predicted = tf.matmul(W_tiled, caps1_output_tiled,name="caps2_predicted")

In [21]:
caps2_predicted

<tf.Tensor 'caps2_predicted:0' shape=(?, 1152, 10, 16, 1) dtype=float32>

# routing by agreement

- "all the routing logits are initialized to zero" in page 4
- $b_{i,j}$ all capsule i in layer l and all capsule j in layer (l+1)
- so, dimension is [caps1_n_caps, caps2_n_caps, 1, 1]

In [59]:
raw_weights = tf.zeros([batch_size, caps1_n_caps, caps2_n_caps, 1, 1],dtype=np.float32, name="raw_weights")

In [30]:
# apply softmax
routing_weights = tf.nn.softmax(raw_weights, dim=2, name="routing_weights")

calculate $\mathbf{s}_j = \sum\limits_{i}{c_{i,j}\hat{\mathbf{u}}_{j|i}}$ (equation (2)-left in the paper):

In [60]:
# multiply
weighted_predictions = tf.multiply(routing_weights, caps2_predicted,name="weighted_predictions")
# sum
weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, keep_dims=True,name="weighted_sum")

In [33]:
# apply squash
caps2_output_round_1 = squash(weighted_sum, axis=-2, name="caps2_output_round_1")
# don't understand axis = -2

In [61]:
caps2_output_round_1_tiled = tf.tile(caps2_output_round_1, [1, caps1_n_caps, 1, 1, 1],name="caps2_output_round_1_tiled")

agreement is simply the scalar product $a_{ij} = \mathbf{v}_{j} \cdot \hat{\mathbf{u}}_{j|i}$  

In [62]:
agreement = tf.matmul(caps2_predicted, caps2_output_round_1_tiled,transpose_a=True, name="agreement")

calculate that $b_{i,j} \gets b_{i,j} + \hat{\mathbf{u}}_{j|i} \cdot \mathbf{v}_j$ 

In [63]:
raw_weights_round_2 = tf.add(raw_weights, agreement,name="raw_weights_round_2")

repeat this Routing

In [64]:
routing_weights_round_2 = tf.nn.softmax(raw_weights_round_2, dim=2, name="routing_weights_round_2")
weighted_predictions_round_2 = tf.multiply(routing_weights_round_2, caps2_predicted, name="weighted_predictions_round_2")
weighted_sum_round_2 = tf.reduce_sum(weighted_predictions_round_2, axis=1, keep_dims=True, name="weighted_sum_round_2")
caps2_output_round_2 = squash(weighted_sum_round_2, axis=-2,name="caps2_output_round_2")

In [39]:
caps2_output = caps2_output_round_2

# Estimated Class Probabilities (Length)
- like above squash fuction , sum epsilon to prevent zero

In [56]:
def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False, name=None):
    with tf.name_scope(name, default_name="safe_norm"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,keep_dims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)

In [46]:
# apply safr_norm to caps2_output / 
# find highest probability by argmax_function
# reduce dimension(size 1) by squeeze function
y_proba = safe_norm(caps2_output, axis=-2, name="y_proba")
y_proba_argmax = tf.argmax(y_proba, axis=2, name="y_proba")
y_pred = tf.squeeze(y_proba_argmax, axis=[1,2], name="y_pred")

# margin loss for digit existence

$ L_k = T_k \max(0, m^{+} - \|\mathbf{v}_k\|)^2 + \lambda (1 - T_k) \max(0, \|\mathbf{v}_k\| - m^{-})^2$

in this papaer, setted parameter 
$m^{+} = 0.9$, $m^{-} = 0.1$ and $\lambda = 0.5$.

and to implement $T_k$ we need one-hot-encoding

In [47]:
m_plus = 0.9
m_minus = 0.1
lambda_ = 0.5

In [48]:
# to transrom by one-hot-encoding
T = tf.one_hot(y, depth=caps2_n_caps, name="T")

In [52]:
# to input caps2_output apply norm function
caps2_output_norm = safe_norm(caps2_output, axis=-2, keep_dims=True,name="caps2_output_norm")

calculate $\max(0, m^{+} - \|\mathbf{v}_k\|)^2$

In [53]:
present_error_raw = tf.square(tf.maximum(0., m_plus - caps2_output_norm),name="present_error_raw")
present_error = tf.reshape(present_error_raw, shape=(-1, 10),name="present_error")
# 10 class , so set the shape (-1,10) -> -1 is auto setting

calculate $\max(0, \|\mathbf{v}_k\| - m^{-})^2$

In [65]:
absent_error_raw = tf.square(tf.maximum(0., caps2_output_norm - m_minus), name="absent_error_raw")
absent_error = tf.reshape(absent_error_raw, shape=(-1, 10), name="absent_error")

In [66]:
# sum and reduce_mean
L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error,name="L")
margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="margin_loss")

# Reconstruction

# Mask
- During training, we mask out all but the activity vector of thr correct digit capsule(page4)
- i can't fullly understand mask means and algorithm.

In [68]:
# set initial of mask
mask_with_labels = tf.placeholder_with_default(False, shape=(), name="mask_with_labels")

# 
reconstruction_targets = tf.cond(mask_with_labels, # condition
                                 lambda: y,        # if True
                                 lambda: y_pred,   # if False
                                 name="reconstruction_targets")
# one-hot-encoding
reconstruction_mask = tf.one_hot(reconstruction_targets, depth=caps2_n_caps, name="reconstruction_mask")

# reshape
reconstruction_mask_reshaped = tf.reshape(reconstruction_mask, [-1, 1, caps2_n_caps, 1, 1], name="reconstruction_mask_reshaped")

# Multiply
caps2_output_masked = tf.multiply(caps2_output, reconstruction_mask_reshaped,name="caps2_output_masked")

# decorder input
decoder_input = tf.reshape(caps2_output_masked,[-1, caps2_n_caps * caps2_n_dims],name="decoder_input")

# Decoder
in this page4 of paper, set the number of output 512, 1024, 784
and activation relu, relu, sigmoid

In [69]:
with tf.name_scope("decoder"):
    hidden1 = tf.layers.dense(decoder_input, 512, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, 1024, activation=tf.nn.relu, name="hidden2")
    decoder_output = tf.layers.dense(hidden2, 784, activation=tf.nn.sigmoid, name="decoder_output")

# Reconstruction loss
- square difference input image vs reconstructed image

In [71]:
X_flat = tf.reshape(X, [-1, 784], name="X_flat")
squared_difference = tf.square(X_flat - decoder_output, name="squared_difference")
reconstruction_loss = tf.reduce_sum(squared_difference, name="reconstruction_loss")

# final loss

In [73]:
alpha = 0.0005 # page4
loss = tf.add(margin_loss, alpha * reconstruction_loss, name="loss")

# Accuracy

In [74]:
correct = tf.equal(y, y_pred, name="correct")
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

# Training operations

In [75]:
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss, name="training_op")

# Training and Test
Training and test is carried out in the same way as implementation of traditional deep-learning.

# Further more
- I notice that Lack of concepts and implement for dimensional conversion
(like as : tf.tile, tf.expand_dims)


- Combining mathematics with coding requires habit.
: If I had implemented it myself, I would have had a hard time thinking about adding eplison to squash.


- study direction
: It was a good opportunity to read the paper while comparing the lines of coding.
  It could tell what to study from now on.<br/>